In [1]:
import pandas as pd
import requests
import time

In [2]:
# Загрузим исходную таблицу
repos = pd.read_csv('https://raw.githubusercontent.com/AndreyG75/AI-DeepFund_Ethereum/refs/heads/main/Contest_2_POND/Datasets/data_contest_all_repos.csv')

In [3]:
# GitHub токен
GITHUB_TOKEN = "GITHUB_TOKEN"
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}

In [11]:
# Функция для извлечения данных о языках из GitHub API
def get_repo_languages(repo):
    try:
        # Извлечение {owner}/{repo} из URL
        repo_path = "/".join(repo.split("/")[-2:])
        api_url = f"https://api.github.com/repos/{repo_path}/languages"

        response = requests.get(api_url, headers=HEADERS)

        # Обработка ошибок
        if response.status_code == 403:
            print(f"Rate limit exceeded or forbidden access for {repo_url}. Retrying after a pause...")
            time.sleep(10)
            return None
        elif response.status_code == 404:
            print(f"Repository {repo_url} not found.")
            return None
        elif response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching data for {repo_url}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception: {e}")
        return None

In [12]:
# Функция для обработки строки и добавления информации о языках
def add_github_languages(row):
    repo_url = row['repo']
    languages_data = get_repo_languages(repo_url)

    if not languages_data:
        return []

    # Вычислим общий процент, чтобы найти относительные значения
    total_bytes = sum(languages_data.values())
    languages_filtered = []

    for lang, bytes_count in languages_data.items():
        percentage = (bytes_count / total_bytes) * 100
        if percentage > 5.0:
            languages_filtered.append({"repo_url": repo_url, "languages": lang, "%": round(percentage, 2)})

    return languages_filtered

In [13]:
# Применение функции ко всей таблице и преобразование в итоговый DataFrame
all_languages = []

for _, row in repos.iterrows():
    languages_info = add_github_languages(row)
    all_languages.extend(languages_info)

In [14]:
# Преобразуем результаты в DataFrame
result_df = pd.DataFrame(all_languages)

In [15]:
# Сохраняем результаты в CSV файл
result_df.to_csv('deepfund_metric_languages.csv', index=False)

In [16]:
# Вывод первых строк результата для проверки
print(result_df.head())

                                   repo_url   languages       %
0          https://github.com/mochajs/mocha  JavaScript   99.03
1        https://github.com/chzyer/readline          Go  100.00
2            https://github.com/gulpjs/gulp  JavaScript  100.00
3        https://github.com/webpack/webpack  JavaScript   97.62
4  https://github.com/redux-saga/redux-saga  JavaScript   82.37


# НАЗНАЧЕНИЕ ВЕСОВ ЯЗЫКАМ ПРОГРАММИРОВАНИЯ 

In [2]:
# Загрузим исходную таблицу
languages = pd.read_csv('https://raw.githubusercontent.com/AndreyG75/AI-DeepFund_Ethereum/refs/heads/main/Contest_2_POND/Datasets/deepfund_metric_languages.csv')

In [3]:
# Создаем пустое множество для уникальных значений
unique_languages = set()

# Пройдемся по каждому значению в столбце 'languages' и разделим по запятой
languages['languages'].dropna().apply(lambda x: unique_languages.update(x.split(',')))

# Преобразуем множество в отсортированный список
unique_languages = sorted(unique_languages)

# Выведем результат
print(unique_languages)

['Assembly', 'C', 'C#', 'C++', 'CSS', 'Cython', 'Go', 'HTML', 'Java', 'JavaScript', 'MDX', 'Nim', 'Objective-C++', 'PHP', 'Python', 'Rust', 'Shell', 'Solidity', 'Starlark', 'Swift', 'TypeScript', 'Yul']


In [4]:
# Опционально можно вывести список в виде строк, каждая на новой строке
for lang in unique_languages:
    print(lang)

Assembly
C
C#
C++
CSS
Cython
Go
HTML
Java
JavaScript
MDX
Nim
Objective-C++
PHP
Python
Rust
Shell
Solidity
Starlark
Swift
TypeScript
Yul


In [5]:
# Подсчет количества уникальных языков
languages_count = len(unique_languages)

# Вывод количества языков
print(f"Количество уникальных языков программирования: {languages_count}")

Количество уникальных языков программирования: 22


In [6]:
# Словарь с весами языков программирования
language_weights = {
    'Assembly': 85,
    'C': 80,
    'C#': 55,
    'C++': 65,
    'CSS': 30,
    'Cython': 50,
    'Go': 75,
    'HTML': 30,
    'Java': 60,
    'JavaScript': 50,
    'MDX': 35,
    'Nim': 45,
    'Objective-C++': 65,
    'PHP': 40,
    'Python': 70,
    'Rust': 80,
    'Shell': 45,
    'Solidity': 100,
    'Starlark': 35,
    'Swift': 40,
    'TypeScript': 55,
    'Yul': 90
}

In [8]:
# Функция для вычисления суммарного веса по списку языков
def calculate_weight(language_list):
    return sum(language_weights.get(lang.strip(), 0) for lang in language_list.split(','))

In [9]:
# Добавление нового столбца "weight" в таблицу
languages['weight'] = languages['languages'].apply(lambda x: calculate_weight(x) if pd.notna(x) else 0)

In [10]:
# Вывод первых строк таблицы для проверки
print(languages.head())

                                   repo_url   languages       %  weight
0          https://github.com/mochajs/mocha  JavaScript   99.03      50
1        https://github.com/chzyer/readline          Go  100.00      75
2            https://github.com/gulpjs/gulp  JavaScript  100.00      50
3        https://github.com/webpack/webpack  JavaScript   97.62      50
4  https://github.com/redux-saga/redux-saga  JavaScript   82.37      50


In [11]:
# Добавление столбца "score" с расчетом
languages['score'] = (languages['%'] * languages['weight'] / 100).round().astype(int)

In [12]:
print(languages.head())

                                   repo_url   languages       %  weight  score
0          https://github.com/mochajs/mocha  JavaScript   99.03      50     50
1        https://github.com/chzyer/readline          Go  100.00      75     75
2            https://github.com/gulpjs/gulp  JavaScript  100.00      50     50
3        https://github.com/webpack/webpack  JavaScript   97.62      50     49
4  https://github.com/redux-saga/redux-saga  JavaScript   82.37      50     41


In [13]:
# Сохранение результата в новый CSV 
languages.to_csv('deepfund_metric_languages_calc.csv', index=False)

In [14]:
# Создание новой таблицы languages_score с нужными столбцами
languages_score = languages[['repo_url', 'score']].copy()

In [15]:
# Переименование столбца repo_url в repo
languages_score.rename(columns={'repo_url': 'repo'}, inplace=True)

In [16]:
# Группировка по столбцу 'repo' и суммирование значений score
languages_score = languages_score.groupby('repo', as_index=False)['score'].sum()

In [17]:
# Вывод таблицы для проверки
print(languages_score.head())

                                           repo  score
0          https://github.com/ajv-validator/ajv     54
1  https://github.com/alexeyraspopov/picocolors     50
2              https://github.com/alloy-rs/core     81
3                https://github.com/axios/axios     50
4                https://github.com/babel/babel     53


In [18]:
# Сохранение результата в файл 
languages_score.to_csv('deepfunding_metric_languages_score.csv', index=False)

In [19]:
# Вывод количества строк с использованием свойства shape
print("Количество строк:", languages_score.shape[0])  # альтернатива len(languages_score)

Количество строк: 117
